In [ ]:
# Импорт необходимых библиотек
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к src
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))

# Прямой импорт из модулей
from src.model_utils import load_model_and_tokenizer, autocomplete_text
from src.data_utils import prepare_test_data, prepare_test_cases
from src.evaluation_utils import test_autocomplete_performance
from src.evaluation_utils import calculate_rouge_scores
from src.evaluation_utils import print_results_analysis 
from src.evaluation_utils import plot_rouge_scores
from src.evaluation_utils import save_results_to_csv


# Проверка устройства
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

# КОНФИГУРАЦИЯ ПАРАМЕТРОВ
config = {
    # Параметры модели
    'model_name': "distilgpt2",
    'max_length': 1024,
    'max_new_tokens': 50,
    'temperature': 0.7,
    'top_k': 50,
    'do_sample': True,
    
    # Параметры данных
    'data_path': "./data/dataset_processed.csv",
    'test_size': 0.5,
    'max_samples': 1000,
    'split_ratio': 0.7,
    'min_length': 20,
    'random_state': 42,
    
    # Параметры оценки
    'metrics': ['rouge1', 'rouge2'],
    'use_stemmer': True,
    
    # Параметры сохранения
    'save_results': True,
    'results_dir': "./results",
    'results_file': "autocomplete_results.csv",
    'metrics_file': "autocomplete_metrics.csv",
    'plot_file': "rouge_metrics_autocomplete.png",
    
    # Параметры тестирования
    'num_test_samples': 500
}

def update_config(**kwargs):
    """Обновление конфигурации"""
    config.update(kwargs)
    print("Configuration updated:")
    for key, value in kwargs.items():
        print(f"  {key}: {value}")

def run_complete_testing():
    """Полный pipeline тестирования"""
    print("Starting complete autocomplete testing...")
    print("Current configuration:")
    for key, value in config.items():
        if key not in ['data_path', 'results_dir']:  # Не показываем длинные пути
            print(f"  {key}: {value}")
    
    # 1. Загрузка модели
    model, tokenizer, device = load_model_and_tokenizer(config['model_name'])
    
    # 2. Подготовка данных
    test_texts = prepare_test_data(
        csv_path=config['data_path'],
        test_size=config['test_size'],
        max_samples=config['max_samples'],
        random_state=config['random_state']
    )
    
    test_cases = prepare_test_cases(
        test_texts,
        split_ratio=config['split_ratio'],
        min_length=config['min_length']
    )
    
    # 3. Тестирование
    results, all_predictions, all_references = test_autocomplete_performance(
        model, tokenizer, device, test_cases,
        num_samples=config['num_test_samples'],
        max_length=config['max_length'],
        max_new_tokens=config['max_new_tokens'],
        temperature=config['temperature'],
        top_k=config['top_k'],
        do_sample=config['do_sample']
    )
    
    # 4. Расчет метрик
    rouge_scores = calculate_rouge_scores(
        all_references, all_predictions,
        metrics=config['metrics'],
        use_stemmer=config['use_stemmer']
    )
    
    # 5. Анализ результатов
    print_results_analysis(results, rouge_scores)
    
    # 6. Визуализация
    plot_rouge_scores(
        rouge_scores,
        save_plot=config['save_results'],
        results_dir=config['results_dir'],
        plot_file=config['plot_file']
    )
    
    # 7. Сохранение
    save_results_to_csv(
        results, rouge_scores,
        save_results=config['save_results'],
        results_dir=config['results_dir'],
        results_file=config['results_file'],
        metrics_file=config['metrics_file']
    )
    
    return results, rouge_scores


In [ ]:

# Запуск тестирования с конфигурацией по умолчанию
print("Запуск с конфигурацией по умолчанию:")
results, rouge_scores = run_complete_testing()


In [ ]:

#print("\n" + "="*50)
#print("ИЗМЕНЕНИЕ КОНФИГУРАЦИИ ДЛЯ СЛЕДУЮЩЕГО ЗАПУСКА")
#print("="*50)

update_config(
    max_new_tokens=50,
    temperature=0.7,
    max_samples=2000,
    num_test_samples=1000,
    test_size= 0.5
)

# Запуск с новыми параметрами
print("\nЗапуск с обновленной конфигурацией:")
results, rouge_scores = run_complete_testing()


In [ ]:
# Быстрый запуск без сохранения результатов
update_config(
    max_samples=100,
    num_test_samples=50,
    test_size= 0.5
    save_results=False
)

print("\nБыстрое тестирование:")
results, rouge_scores = run_complete_testing()